## Aprendizado de Máquina Federado
### Teste local do tensorflow_federated com simulação local de dispositivos de borda

In [7]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

In [8]:
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [9]:
# 1. Carregar o dataset Federado
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

# 2. Definir as funções de pré-processamento
NUM_CLIENTS = 10
BATCH_SIZE = 20
PREFETCH_BUFFER = 10

def preprocess(dataset):
    """Pré-processa o dataset para o modelo Keras."""

    # Mapear imagens para floats de 0-1 e redimensionar para vetores de 784
    def batch_format_fn(element):
        return (tf.reshape(element['pixels'], [-1, 784]),  # Imagens (28*28 = 784)
                tf.reshape(element['label'], [-1, 1]))    # Rótulos

    return dataset.batch(BATCH_SIZE).map(batch_format_fn).repeat(1).prefetch(PREFETCH_BUFFER)

# 3. Aplicar o pré-processamento nos dados de treinamento
# Vamos selecionar apenas um subconjunto de clientes para a simulação
sample_clients = emnist_train.client_ids[0:NUM_CLIENTS]

federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x)) 
                        for x in sample_clients]

In [18]:
def create_keras_model():
    """Cria um modelo Keras simples para classificação MNIST."""
    # O input é um vetor de 784 (28x28) pixels
    initializer = tf.keras.initializers.GlorotNormal(seed=1)
    
    return tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(784,)),
        tf.keras.layers.Dense(10, kernel_initializer=initializer), # 10 classes de saída (0-9)
        tf.keras.layers.Softmax(),
    ])

def model_fn():
    """Compila o modelo Keras e o empacota para TFF."""
    keras_model = create_keras_model()
    
    # O TFF requer um modelo Keras com um Optimizer para treinar localmente
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=federated_train_data[0].element_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [22]:
# CÓDIGO CORRETO PARA TFF v0.87.0 (API Simplificada)

# Construir o processo FedAvg
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    # Otimizador usado por CADA cliente para treinamento local
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    # Otimizador usado pelo SERVIDOR para agregar os pesos
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0) 
)

# Inicializar o estado global do modelo
state = iterative_process.initialize()
print("Modelo Global Inicializado.")

AttributeError: 'function' object has no attribute 'initialize'